In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [40]:
df = pd.read_csv('./deid_reports_for_emel.csv')
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

In [41]:
df.shape

(30135, 6)

In [42]:
df.columns

Index(['report_id', 'clinic_id', 'modality', 'report_text', 'clean_report',
       'structured_report_dict'],
      dtype='object')

In [43]:
df.sample(1)

,report_id,clinic_id,modality,report_text,clean_report,structured_report_dict
21208,6a0718c05ae72d1f85732733c13e99c10946b48b65cb1a...,4a7e321243420b65c96d331efc5140c52563dce27e0868...,MR,MR ABDOMEN WITHOUT IV CONTRAST INCLUDING MRCP\...,MR ABDOMEN WITHOUT IV CONTRAST INCLUDING MRCP ...,{('MR ABDOMEN WITHOUT IV CONTRAST INCLUDING MR...


In [44]:
df['clinic_id'].nunique()

6

In [45]:
df['clinic_id'].value_counts()

clinic_id
3f2b2c77fcfb97463ccf3a792217c8cc7e420428e751b5c623ea55bebe5e33dd    10720
f3f2ef1e-9973-4454-86ce-f9c8d19da7ef                                 8325
bcb4bdd71a574854c688787fa8ca0dbaee668d22d3d031505725aee06fa6ec4d     3451
4921d977174aa6f191ef655af6918f5b028afda936aeb141a0cd9d31537f828b     3398
4a7e321243420b65c96d331efc5140c52563dce27e0868dc59f0f37d28c0780d     1975
cfd6ab910cc116eaa3a8749d43f93c4c0f8f3ef345605647a8b03157fd42ec3a      865
Name: count, dtype: int64

In [46]:
clinic_ids = df['clinic_id'].value_counts().index.tolist()

In [47]:
# clinic_ids
mapped_clinic_ids = {clinic_id: f"clinic_{idx + 1}" for idx, clinic_id in enumerate(clinic_ids)}

In [48]:
mapped_clinic_ids

{'3f2b2c77fcfb97463ccf3a792217c8cc7e420428e751b5c623ea55bebe5e33dd': 'clinic_1',
 'f3f2ef1e-9973-4454-86ce-f9c8d19da7ef': 'clinic_2',
 'bcb4bdd71a574854c688787fa8ca0dbaee668d22d3d031505725aee06fa6ec4d': 'clinic_3',
 '4921d977174aa6f191ef655af6918f5b028afda936aeb141a0cd9d31537f828b': 'clinic_4',
 '4a7e321243420b65c96d331efc5140c52563dce27e0868dc59f0f37d28c0780d': 'clinic_5',
 'cfd6ab910cc116eaa3a8749d43f93c4c0f8f3ef345605647a8b03157fd42ec3a': 'clinic_6'}

In [49]:
df['mapped_clinic_id'] = df['clinic_id'].map(mapped_clinic_ids)

In [50]:
df['modality'].value_counts()

modality
MR     18046
CT      5322
CR      2205
US      2139
XR       879
NM       114
PET        7
PT         4
IR         4
MG         4
FL         4
DF         3
DX         2
MRA        1
Name: count, dtype: int64

In [51]:
df.groupby(['mapped_clinic_id', 'modality']).size().unstack(fill_value=0)

modality,CR,CT,DF,DX,FL,IR,MG,MR,MRA,NM,PET,PT,US,XR
mapped_clinic_id,,,,,,,,,,,,,,
clinic_1,862,33,0,0,0,0,0,9825,0,0,0,0,0,0
clinic_2,822,1932,3,0,0,0,0,4860,1,11,7,0,689,0
clinic_3,289,726,0,2,4,0,3,1377,0,63,0,0,987,0
clinic_4,0,1701,0,0,0,0,0,1693,0,0,0,4,0,0
clinic_5,232,656,0,0,0,4,1,261,0,40,0,0,411,370
clinic_6,0,274,0,0,0,0,0,30,0,0,0,0,52,509


In [ ]:
# Modality Grouping Strategy
#   Positive Effects:
#   1. Cleaner Contextualization: Model learns meaningful modality patterns rather than noise from tiny categories
#   2. Better Generalization: "OTHER" category teaches model to handle rare/unseen modalities
#   3. Reduced Overfitting: Prevents model from memorizing individual rare cases

#   Potential Concerns:
#   1. Lost Specificity: PET scans have very different language than MG, but they'd be grouped together
#   2. Edge Case Handling: Model might be less precise for truly rare modalities

In [56]:
#MR, CT, CR, US, XR, NM -> same name
# All others -> OTHER

modality_mapping = {mod: mod if mod in ["MR", "CT", "CR", "US", "XR", "NM"] else "OTHER" for mod in df['modality'].unique()}

In [57]:
modality_mapping

{'MR': 'MR',
 'CT': 'CT',
 'CR': 'CR',
 'US': 'US',
 'XR': 'XR',
 'NM': 'NM',
 'DX': 'OTHER',
 nan: 'OTHER',
 'PT': 'OTHER',
 'PET': 'OTHER',
 'IR': 'OTHER',
 'DF': 'OTHER',
 'MG': 'OTHER',
 'FL': 'OTHER',
 'MRA': 'OTHER'}

In [58]:
## Mapping modalities
df['mapped_modality'] = df['modality'].map(modality_mapping)

In [59]:
df.groupby(['mapped_clinic_id', 'mapped_modality']).size().unstack(fill_value=0)

mapped_modality,CR,CT,MR,NM,OTHER,US,XR
mapped_clinic_id,,,,,,,
clinic_1,862,33,9825,0,0,0,0
clinic_2,822,1932,4860,11,11,689,0
clinic_3,289,726,1377,63,9,987,0
clinic_4,0,1701,1693,0,4,0,0
clinic_5,232,656,261,40,5,411,370
clinic_6,0,274,30,0,0,52,509


In [52]:
from tqdm import tqdm
import pandas as pd
import ast
import json


def convert_to_json(structured_report):
    try:
        # Handle NaN or non-string inputs
        if pd.isna(structured_report) or not isinstance(structured_report, str):
            return {}

        # Parse string to dictionary
        parsed_dict = ast.literal_eval(structured_report)

        # Convert to cleaner dictionary
        clean_dict = {}
        for key_tuple, value_list in parsed_dict.items():
            # Ensure key is a tuple with one element
            if not isinstance(key_tuple, tuple) or len(key_tuple) != 1:
                continue  # Skip invalid keys
            key = key_tuple[0].rstrip(':').lower().strip()  # Clean key
            # If the list has only one item, convert to string; otherwise, keep as list
            value = value_list[0] if len(value_list) == 1 else value_list
            clean_dict[key] = value

        return clean_dict  # Return dict for efficiency
    except (SyntaxError, ValueError) as e:
        print(f"Error parsing report: {e}")
        return {}

In [55]:
df['report_text'].str.contains('Electronically Signed by', na=False).sum()

3644

In [63]:
sample_data = df['structured_report_dict'].sample(1).values[0]
data = convert_to_json(sample_data)
print(list(data.keys()))
print("=========================\n")
print(f"Findings length: {len(data.get('findings', []))}, Impression length: {len(data.get('impression', []))}\n")
print("=========================\n")
print(data)

['ct abdomen and pelvis without contrast, stone protocol.', 'indication', 'comparison', 'technique', 'findings', 'impression', 'no_header_53']

Findings length: 2, Impression length: 3


{'ct abdomen and pelvis without contrast, stone protocol.': '', 'indication': 'left flank pain, h/o kidney stones.', 'comparison': 'None.', 'technique': 'Routine CT of the abdomen and pelvis was performed without intravenous or oral contrast per renal calculus evaluation protocol. This exam is intended to evaluate for presence of renal calculi and does not adequately assess for renal or urothelial malignancies. Radiation dose reduction was utilized (automated exposure control, mA or kV adjustment based on patient size, or iterative image reconstruction). Coronal and sagittal reformatted images were generated and reviewed.', 'findings': ['Limited assessment of the abdominal organs in the absence of intravenous contrast.', ''], 'impression': ['There is a 5 mm stone in the distal left ureter causing moder

In [47]:
# # Step 1: Get all unique keys from the dictionaries in structured_report_dict
# all_keys = set()
# for report in tqdm(df['structured_report_dict'], desc="Extracting keys"):
#     report_dict = convert_to_json(report)
#     all_keys.update(report_dict.keys())

# # Step 2: Add each key as a column, keeping lists as-is
# for key in tqdm(all_keys, desc="Creating columns"):
#     df[key] = df['structured_report_dict'].apply(
#         lambda x: convert_to_json(x).get(key, None)
#     )

# # Step 3: Split list-valued keys (e.g., IMPRESSION) into separate columns
# # Find the maximum number of items in the IMPRESSION list across all rows
# max_impressions = 0
# for report in tqdm(df['structured_report_dict'], desc="Calculating max impressions"):
#     report_dict = convert_to_json(report)
#     impression = report_dict.get('IMPRESSION', [])
#     if isinstance(impression, list):  # Ensure it's a list
#         max_impressions = max(max_impressions, len(impression))

# # Add separate columns for each IMPRESSION item
# for i in tqdm(range(max_impressions), desc="Creating IMPRESSION columns"):
#     df[f'IMPRESSION_{i+1}'] = df['structured_report_dict'].apply(
#         lambda x: convert_to_json(x).get('IMPRESSION', [])[i]
#         if isinstance(convert_to_json(x).get('IMPRESSION', []), list) and i < len(convert_to_json(x).get('IMPRESSION', []))
#         else None
#     )

# # Optionally drop the original structured_report_dict column
# # df = df.drop('structured_report_dict', axis=1)

# # Save the updated DataFrame
# df.to_csv('updated_deid_reports.csv', index=False)

# # Print updated columns
# print("Updated DataFrame columns:")
# print(df.columns)

Creating columns:   4%|▍         | 99/2456 [03:27<1:21:37,  2.08s/it]/var/folders/xw/88_6_hq95z5fb8b7c8xdtg7m0000gn/T/ipykernel_70384/3256823615.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key] = df['structured_report_dict'].apply(
Creating columns:   4%|▍         | 100/2456 [03:29<1:21:19,  2.07s/it]/var/folders/xw/88_6_hq95z5fb8b7c8xdtg7m0000gn/T/ipykernel_70384/3256823615.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key] = df['structured_report_dict'].apply(
Creating columns:   4%|▍         | 101/2456 [

Updated DataFrame columns:
Index(['report_id', 'clinic_id', 'modality', 'report text', 'clean_report',
       'structured_report_dict',
       'ULTRASOUND ABDOMEN LIMITED RIGHT UPPER QUADRANT',
       'Splenic coronal length', 'Origin', 'Paraspinal tissues',
       ...
       'IMPRESSION_14', 'IMPRESSION_15', 'IMPRESSION_16', 'IMPRESSION_17',
       'IMPRESSION_18', 'IMPRESSION_19', 'IMPRESSION_20', 'IMPRESSION_21',
       'IMPRESSION_22', 'IMPRESSION_23'],
      dtype='object', length=2485)


In [ ]:
# updated_df = pd.read_csv('updated_deid_reports.csv')

In [49]:
# updated_df.shape

(30135, 2485)